In [ ]:
!pip install torchaudio
!pip install librosa
!pip install torch
!pip install scipy
!pip install tqdm
!pip install rotary_embedding_torch
!pip install transformers
!pip install tokenizers
!pip install inflect
!pip install progressbar
!pip install einops
!pip install unidecode
!pip install pyngrok
!pip install tortoise-tts
!pip install -U kaleido
!pip install flask_ngrok
!pip install flask-cors

In [ ]:
!npm install -g localtunnel


added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/Audio detection model/

/content/drive/MyDrive/Colab Notebooks/Audio detection model


In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import torch
import torchaudio
import torch.nn.functional as F
import io
import base64
from io import BytesIO
import matplotlib.pyplot as plt
from pyngrok import ngrok
from tortoise.models.classifier import AudioMiniEncoderWithClassifierHead
from flask_cors import CORS


now here is our model

In [ ]:
app = Flask(__name__)
CORS(app)



# ngrok.set_auth_token("2leiYywqQOXyRcIykX1ZcKfLNQA_5c54v5kRVtLMeGfHqiEYy")
# public_url = ngrok.connect(5000).public_url
# print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000/\"")


# Load audio function
def load_audio(audio_bytes, sampling_rate=22000):
    try:
        # Load the audio from bytes
        audio, lsr = torchaudio.load(io.BytesIO(audio_bytes))

        # If stereo, convert to mono by averaging both channels
        if audio.shape[0] > 1:
            audio = torch.mean(audio, dim=0, keepdim=True)

        if lsr != sampling_rate:
            audio = torchaudio.functional.resample(audio, lsr, sampling_rate)

        audio = torch.clip(audio, -1, 1)  # Clip audio between -1 and 1

        # Make sure the shape is [1, time_steps] (mono, unbatched) before adding batch dim
        return audio  # No unsqueeze here, leave it as [channels, time_steps]
    except Exception as e:
        print(f"Error loading audio: {e}")
        raise

# Audio classification function
def classify_audio_clip(clip):
    try:
        # Define your classifier (replace it with your own model architecture)
        classifier = AudioMiniEncoderWithClassifierHead(2, spec_dim=1, embedding_dim=512, depth=5, downsample_factor=4,
                                                        resnet_blocks=2, attn_blocks=4, num_attn_heads=4, base_channels=32,
                                                        dropout=0, kernel_size=5, distribute_zero_label=False)
        state_dict = torch.load('classifier.pth', map_location=torch.device('cpu'))
        classifier.load_state_dict(state_dict)

        # If the clip is 2D (e.g. [channels, time_steps]), add a batch dimension: [batch, channels, time_steps]
        if clip.dim() == 2:
            clip = clip.unsqueeze(0)

        results = F.softmax(classifier(clip), dim=-1)
        return results[0][0]
    except Exception as e:
        print(f"Error in classification: {e}")
        raise


# Function to generate waveform plot using matplotlib
def generate_waveform_plot(audio_clip):
    try:
        plt.figure(figsize=(10, 4))
        plt.plot(audio_clip.squeeze().numpy())
        plt.title("Waveform Plot")
        plt.xlabel("Time")
        plt.ylabel("Amplitude")

        # Save the plot to a bytes buffer
        buf = BytesIO()
        plt.savefig(buf, format="png")
        buf.seek(0)

        # Encode the bytes as base64
        encoded_image = base64.b64encode(buf.read()).decode('utf-8')
        buf.close()

        # Return the base64-encoded image
        return encoded_image
    except Exception as e:
        print(f"Error generating waveform plot: {e}")
        raise


@app.route('/')
def home():
    return "<h1>Hello, World!</h1>"

@app.route('/predict_audio', methods=['POST'])
def upload_audio():
    try:
        # Check if a file is uploaded
        if 'file' not in request.files:
            return jsonify({'error': 'No file part in the request'}), 400

        file = request.files['file']

        if file.filename == '':
            return jsonify({'error': 'No selected file'}), 400

        if file:
            # Process the audio file here
            audio_bytes = file.read()

            # Load and classify the audio file
            audio_clip = load_audio(audio_bytes)
            result = classify_audio_clip(audio_clip)

            # Generate waveform plot
            waveform_image = generate_waveform_plot(audio_clip)

            # Convert audio file to base64 for returning in JSON
            audio_b64 = base64.b64encode(audio_bytes).decode('utf-8')
            audio_mime = f"data:audio/mp3;base64,{audio_b64}"

            # Return a JSON response containing result, audio, and waveform image
            return jsonify({
                'result': result.item(),
                'audio': audio_mime,
                'waveform_image': waveform_image
            }), 200

        return jsonify({'error': 'File processing failed'}), 500

    except Exception as e:
        print(f"Error in processing the request: {e}")
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(port=5000)

ERROR:pyngrok.process.ngrok:t=2024-09-05T14:44:57+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/tunnels/agents\r\n\r\nERR_NGROK_108\r\n"
ERROR:pyngrok.process.ngrok:t=2024-09-05T14:44:57+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/tunnels/agents\r\n\r\nERR_NGROK_108\r\n.

In [ ]:

# def load_audio(audiopath, sampling_rate=22000):
#     if isinstance(audiopath, str): # If the input is a file path
#         if audiopath.endswith('.mp3'):
#             audio, lsr = librosa.load(audiopath, sr=sampling_rate)
#             audio = torch. FloatTensor(audio)
#         else:
#             assert False, f"Unsupported audio format provided: {audiopath[-4:]}"
#     elif isinstance(audiopath, io.BytesIO): # If the input is file content
#         audio, lsr = torchaudio.load(audiopath)
#         audio = audio[0]

#     if lsr != sampling_rate:
#         audio = torchaudio. functional.resample(audio, lsr, sampling_rate)

#     if torch.any(audio > 2) or not torch.any(audio < 0):
#         print(f"Error with audio data. Max={audio.max()} min={audio.min()}")
#     audio.clip (-1, 1)

#     return audio.unsqueeze(0)

# def classify_audio_clip(clip):

#     '''
#     Returns whether or not the classifier thinks the given clip came from AI generation.
#     : param clip: torch tensor containing audio waveform data (get it from load_audio)
#     :return: The probability of the audio clip being AI-generated.
#     '''
#     classifier = AudioMiniEncoderWithClassifierHead(2, spec_dim=1, embedding_dim=512, depth=5, downsample_factor=4,
#                                                     resnet_blocks=2, attn_blocks=4, num_attn_heads=4, base_channels=32,
#                                                     dropout=0, kernel_size=5, distribute_zero_label=False)
#     state_dict = torch.load('classifier.pth', map_location=torch.device('cpu'))
#     classifier.load_state_dict(state_dict)
#     clip = clip.cpu().unsqueeze(0)
#     results = F.softmax(classifier(clip), dim =- 1)
#     return results [0][0]

# st.set_page_config(Layout="wide")

# def main():
#     st.title("audio detector")
#     uploaded_file = st.file_uploader("Upload an audio file", type=["mp3"])

#     if uploaded_file is not None:
#         if st.button("Analyze Audio"):
#             col1, col2, col3=st.columns(3)

#             with col1:
#                 st.info("Your results are below")
#                 #Load and classify the audio file
#                 audio_clip = load_audio(uploaded_file)
#                 result = classify_audio_clip(audio_clip)
#                 result = result.item()
#                 st.info("Result Probability: {result}")
#                 st.success(f"The uploaded audio is {result * 100: .2f}% likely to be AI Generated.")

#             with col2:
#                 st.info("Your uploaded audio is below")
#                 st.audio(uploaded_file)
#                 #create a waveform
#                 fig = px. line()
#                 fig.add_scatter(x=list(range(len(audio_clip.squeeze()))), y=audio_clip.squeeze())
#                 fig.update_layout(
#                     title="Waveform Plot",
#                     xaxis_title = "Time",
#                     yaxis_title="Amplitude")
#                 st.plotly_chart(fig, use_container_width=True)

#             with col3:
#                 st.info("Disclaimer")
#                 st.warning("These classification or detection mechanisms are not always accurate. They should be considered as a strong signal and not the ultimate decision makers.")



# if __name__ == "__main__":
#     main()

Overwriting audio_classifier.py
